In [1]:
# from qpy.gates import H

import numpy as np

from qpy import *

# from qpy.linalg import *
# from qpy.gates import *
# from qpy.measurement import *
import main


In [2]:
#toffoli gate 
def TOFFOLI(control1, control2, target, num_qubits):
    
    if control1 == target or control2 == target:
        raise ValueError("Control and Target qubits must be different.")
    
    if not (0<= control1 <= num_qubits) or not (0<= target <= num_qubits) or not (0<= control2 <= num_qubits):
        raise ValueError("Control and target qubits must be within range")
        
    # Determine the order of applying control qubits based on their indices
    c1, c2 = sorted([control1, control2])  # Ensure control_qubits are ordered in ascending order

    # Apply the controlled NOT gate with padding
    if c1 < target:   # If both controls are before the target
        operator = apply_kron([*[I]*(c2), np.outer(zero,zero), *[I]*(num_qubits - c2 - 1)]) + apply_kron([*[I]*(c1), np.outer(zero,zero), *[I]*(num_qubits - c1 - 1)]) + apply_kron([*[I]*(c2), np.outer(one,one), *[I]*(c1 - c2 - 1), np.outer(one,one), *[I]*(target - c1 - 1), X, *[I]*(num_qubits - target - 1)])
    
    elif c2 < target < c1: # If target qubit is in the middle of both control
        operator = apply_kron([*[I]*(2), np.outer(zero,zero), *[I]*(num_qubits - c2 - 1)]) + apply_kron([*[I]*(c1), np.outer(zero,zero), *[I]*(num_qubits - c1 - 1)]) + apply_kron([*[I]*(c2), np.outer(one,one), *[I]*(target - c2 - 1), X, *[I]*(c1 - target - 1), np.outer(one,one), *[I]*(num_qubits - c1 - 1)])
        
    elif target < c2: # if target is before both control
        operator = apply_kron([*[I]*(2), np.outer(zero,zero), *[I]*(num_qubits - c2 - 1)]) + apply_kron([*[I]*(c1), np.outer(zero,zero), *[I]*(num_qubits - c1 - 1)]) + apply_kron([*[I]*(target), X, *[I]*(c2 - target - 1), np.outer(one,one), *[I]*(c1 - c2 - 1), np.outer(one,one), *[I]*(num_qubits - c1 - 1)])
        
    return operator


In [3]:
#generating a bell pair
# def bell_pair(q1, q2):
#     bell_state = np.array([1,0,0,0])
#     bell_state = np.dot(np.diag([1,1,1,1]))
#     bell_state = np.dot(cnot, bell_state)
    
#     return bell_state


In [4]:
# bell = bell_pair(1,2)
# print(bell)

In [16]:
def bell_pair():
    bell_state = np.kron(zero,zero)
    bell_state = np.dot(np.kron(H, I), bell_state)
    bell_state = np.dot(cnot, bell_state)
    
    return bell_state

print(bell_pair())

def teleport_quantum_state(original_state):
    """Teleports a quantum state using an entangled Bell state."""
    num_qubits = len(original_state)
    bell_state = bell_pair()

    state = np.kron(original_state, bell_state)
    
    # Apply CNOT controlled by original state
    state = cnot_nonadj(0, 1, 3)@state
    # print(state)

    # Apply Hadamard to controlled qubit
    state = apply_gate(H, 0, 3)@state
    # print(state)

    # Measure both control qubits
    state, outcome1 = measure(state, 0)
    # print(state)
    
    state, outcome2 = measure(state, 2)
    # print(state)

    # Apply classical communication and corrections
    classical_bits = str(outcome2) + str(outcome1)
    correction_map = {
        "00": np.eye(2),
        "01": X,
        "10": Z,
        "11": Y
    }

    correction = apply_gate(correction_map[classical_bits], 2, 3)
    state = np.dot(correction, state)
    # print(state)

    return state

[0.70710678 0.         0.         0.70710678]


In [22]:
stt = teleport_quantum_state([1,0])
print(stt)

[0. 0. 0. 0. 0. 1. 0. 0.]


In [ ]:
%debug

> c:\users\raisa\onedrive\documents\qpy\qpy\measurement.py(22)measure()
     20     # P1 = np.real(np.trace(M1 @ state))
     21 
---> 22     assert np.isclose(P0 + P1, 1)
     23 
     24     if np.random.rand()<P0:

array([[ 1.00000000e+00,  0.00000000e+00, -2.23711432e-17,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [-2.23711432e-17,  0.00000000e+00,  1.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00]])
*** NameError: name 'p1' is not defined
array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
        -2.23711432e-17],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00, -2.23711432e-17,  0.00000000e+00,
         1.00000000e+00]])
array([[ 0.70710678,  0.        ,  0.70710678,  0.        ],